<a href="https://colab.research.google.com/github/nanfang-wuyu/ML4NLP1_UZH/blob/main/Assignment%202/word_embeddings_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

Source: [https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#exercise-computing-word-embeddings-continuous-bag-of-words](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#exercise-computing-word-embeddings-continuous-bag-of-words)

# Word Embeddings: Encoding Lexical Semantics

Word embeddings are dense vectors of real numbers, one per word in your
vocabulary. In NLP, it is almost always the case that your features are
words! But how should you represent a word in a computer? You could
store its ascii character representation, but that only tells you what
the word *is*, it doesn't say much about what it *means* (you might be
able to derive its part of speech from its affixes, or properties from
its capitalization, but not much). Even more, in what sense could you
combine these representations? We often want dense outputs from our
neural networks, where the inputs are $|V|$ dimensional, where
$V$ is our vocabulary, but often the outputs are only a few
dimensional (if we are only predicting a handful of labels, for
instance). How do we get from a massive dimensional space to a smaller
dimensional space?

How about instead of ascii representations, we use a one-hot encoding?
That is, we represent the word $w$ by

\begin{align}\overbrace{\left[ 0, 0, \dots, 1, \dots, 0, 0 \right]}^\text{|V| elements}\end{align}

where the 1 is in a location unique to $w$. Any other word will
have a 1 in some other location, and a 0 everywhere else.

There is an enormous drawback to this representation, besides just how
huge it is. It basically treats all words as independent entities with
no relation to each other. What we really want is some notion of
*similarity* between words. Why? Let's see an example.

Suppose we are building a language model. Suppose we have seen the
sentences

* The mathematician ran to the store.
* The physicist ran to the store.
* The mathematician solved the open problem.

in our training data. Now suppose we get a new sentence never before
seen in our training data:

* The physicist solved the open problem.

Our language model might do OK on this sentence, but wouldn't it be much
better if we could use the following two facts:

* We have seen  mathematician and physicist in the same role in a sentence. Somehow they
  have a semantic relation.
* We have seen mathematician in the same role  in this new unseen sentence
  as we are now seeing physicist.

and then infer that physicist is actually a good fit in the new unseen
sentence? This is what we mean by a notion of similarity: we mean
*semantic similarity*, not simply having similar orthographic
representations. It is a technique to combat the sparsity of linguistic
data, by connecting the dots between what we have seen and what we
haven't. This example of course relies on a fundamental linguistic
assumption: that words appearing in similar contexts are related to each
other semantically. This is called the `distributional
hypothesis <https://en.wikipedia.org/wiki/Distributional_semantics>`__.


# Getting Dense Word Embeddings

How can we solve this problem? That is, how could we actually encode
semantic similarity in words? Maybe we think up some semantic
attributes. For example, we see that both mathematicians and physicists
can run, so maybe we give these words a high score for the "is able to
run" semantic attribute. Think of some other attributes, and imagine
what you might score some common words on those attributes.

If each attribute is a dimension, then we might give each word a vector,
like this:

\begin{align}q_\text{mathematician} = \left[ \overbrace{2.3}^\text{can run},
   \overbrace{9.4}^\text{likes coffee}, \overbrace{-5.5}^\text{majored in Physics}, \dots \right]\end{align}

\begin{align}q_\text{physicist} = \left[ \overbrace{2.5}^\text{can run},
   \overbrace{9.1}^\text{likes coffee}, \overbrace{6.4}^\text{majored in Physics}, \dots \right]\end{align}

Then we can get a measure of similarity between these words by doing:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = q_\text{physicist} \cdot q_\text{mathematician}\end{align}

Although it is more common to normalize by the lengths:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = \frac{q_\text{physicist} \cdot q_\text{mathematician}}
   {\| q_\text{\physicist} \| \| q_\text{mathematician} \|} = \cos (\phi)\end{align}

Where $\phi$ is the angle between the two vectors. That way,
extremely similar words (words whose embeddings point in the same
direction) will have similarity 1. Extremely dissimilar words should
have similarity -1.


You can think of the sparse one-hot vectors from the beginning of this
section as a special case of these new vectors we have defined, where
each word basically has similarity 0, and we gave each word some unique
semantic attribute. These new vectors are *dense*, which is to say their
entries are (typically) non-zero.

But these new vectors are a big pain: you could think of thousands of
different semantic attributes that might be relevant to determining
similarity, and how on earth would you set the values of the different
attributes? Central to the idea of deep learning is that the neural
network learns representations of the features, rather than requiring
the programmer to design them herself. So why not just let the word
embeddings be parameters in our model, and then be updated during
training? This is exactly what we will do. We will have some *latent
semantic attributes* that the network can, in principle, learn. Note
that the word embeddings will probably not be interpretable. That is,
although with our hand-crafted vectors above we can see that
mathematicians and physicists are similar in that they both like coffee,
if we allow a neural network to learn the embeddings and see that both
mathematicians and physicists have a large value in the second
dimension, it is not clear what that means. They are similar in some
latent semantic dimension, but this probably has no interpretation to
us.


In summary, **word embeddings are a representation of the *semantics* of
a word, efficiently encoding semantic information that might be relevant
to the task at hand**. You can embed other things too: part of speech
tags, parse trees, anything! The idea of feature embeddings is central
to the field.


# Word Embeddings in Pytorch

Before we get to a worked example and an exercise, a few quick notes
about how to use embeddings in Pytorch and in deep learning programming
in general. Similar to how we defined a unique index for each word when
making one-hot vectors, we also need to define an index for each word
when using embeddings. These will be keys into a lookup table. That is,
embeddings are stored as a $|V| \times D$ matrix, where $D$
is the dimensionality of the embeddings, such that the word assigned
index $i$ has its embedding stored in the $i$'th row of the
matrix. In all of my code, the mapping from words to indices is a
dictionary named word\_to\_ix.

The module that allows you to use embeddings is torch.nn.Embedding,
which takes two arguments: the vocabulary size, and the dimensionality
of the embeddings.

To index into this table, you must use torch.LongTensor (since the
indices are integers, not floats).




In [79]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [80]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings # randomly initial
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[ 0.6614,  0.2669,  0.0617,  0.6213, -0.4519]],
       grad_fn=<EmbeddingBackward0>)


In [81]:
embeds(torch.LongTensor([1]))

tensor([[-0.1661, -1.5228,  0.3817, -1.0276, -0.5631]],
       grad_fn=<EmbeddingBackward0>)

# An Example: N-Gram Language Modeling

Recall that in an n-gram language model, given a sequence of words
$w$, we want to compute

\begin{align}P(w_i | w_{i-1}, w_{i-2}, \dots, w_{i-n+1} )\end{align}

Where $w_i$ is the ith word of the sequence.

In this example, we will compute the loss function on some training
examples and update the parameters with backpropagation.




In [82]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # n probs, 1 target idx
        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]
[520.4369215965271, 517.8524951934814, 515.2855088710785, 512.7362174987793, 510.2031500339508, 507.683557510376, 505.17774844169617, 502.6867473125458, 500.20881819725037, 497.74234104156494]


# Exercise: Computing Word Embeddings: Continuous Bag-of-Words

The Continuous Bag-of-Words model (CBOW) is frequently used in NLP deep
learning. It is a model that tries to predict words given the context of
a few words before and a few words after the target word. This is
distinct from language modeling, since CBOW is not sequential and does
not have to be probabilistic. Typcially, CBOW is used to quickly train
word embeddings, and these embeddings are used to initialize the
embeddings of some more complicated model. Usually, this is referred to
as *pretraining embeddings*. It almost always helps performance a couple
of percent.

The CBOW model is as follows. Given a target word $w_i$ and an
$N$ context window on each side, $w_{i-1}, \dots, w_{i-N}$
and $w_{i+1}, \dots, w_{i+N}$, referring to all context words
collectively as $C$, CBOW tries to minimize

\begin{align}-\log p(w_i | C) = -\log \text{Softmax}(A(\sum_{w \in C} q_w) + b)\end{align}

where $q_w$ is the embedding for word $w$.

Implement this model in Pytorch by filling in the class below. Some
tips:

* Think about which parameters you need to define.
* Make sure you know what shape each operation expects. Use .view() if you need to
  reshape.




In [83]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])




def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

# data[i]: i-th contexts-target pair
# data[i][0]: 4 contexts (list)
# data[i][1]: target (text)
make_context_vector(data[0][0], word_to_ix)  # example

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


tensor([40, 27,  9,  4])

## Model

In [7]:
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(2 * context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)


    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

# create your model and train.  here are some functions to help you make
# the data ready for use by your module


## Data Preprocessing

In [9]:
!gdown 1foE1JuZJeu5E_4qVge9kExzhvF32teuF
!gdown 13IWXrTjGTrfCd9l7dScZVO8ZvMicPU75

Downloading...
From: https://drive.google.com/uc?id=1foE1JuZJeu5E_4qVge9kExzhvF32teuF
To: /content/tripadvisor_hotel_reviews_reduced.csv
100% 7.36M/7.36M [00:00<00:00, 74.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=13IWXrTjGTrfCd9l7dScZVO8ZvMicPU75
To: /content/scifi_reduced.txt
100% 43.1M/43.1M [00:00<00:00, 55.2MB/s]


### Load Data

In [10]:
with open(f'scifi_reduced.txt') as f:
    text_scifi = f.read()
import pandas as pd
df_hotel = pd.read_csv('tripadvisor_hotel_reviews_reduced.csv')

In [11]:
text_scifi[:20]

' A chat with the edi'

In [12]:
df_hotel[:5]

,Review,Rating
0,fantastic service large hotel caters business ...,5
1,"great hotel modern hotel good location, locate...",4
2,3 star plus glasgowjust got 30th november 4 da...,4
3,nice stayed hotel nov 19-23. great little bout...,4
4,great place wonderful hotel ideally located me...,5


In [13]:
text_hotel = df_hotel.drop(columns=["Rating"])

In [14]:
text_hotel[:5]

,Review
0,fantastic service large hotel caters business ...
1,"great hotel modern hotel good location, locate..."
2,3 star plus glasgowjust got 30th november 4 da...
3,nice stayed hotel nov 19-23. great little bout...
4,great place wonderful hotel ideally located me...


1. Data Cleaning, 2. **Character Casing**, 3. **Splitting Sentences** (optional), 4. **Tokenizing Words**, 5. **Dealing with Punctuation**, 6. Expanding Contractions (optional), 7. **Removing Stopwords**, 8. Lemmatization and Stemming (optional), 9. Handling Special Cases, 10. Handling Rare Words and Out-of-Vocabulary Words.

1. Special Characters Cleaning
2. Character Casing
3. Do not split sentences or tokenize words
4. Do not remove stopword

### Special Characters Cleaning

In [15]:
# import library: Regular Expression
import re

# Clean the data by removing special characters (punctuation)
def sp_chara_cleaning(text):
    clean_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    return clean_text

In [16]:
sp_chara_cleaning("#trap music <123> @gmail! How are you?")

'trap music 123 gmail How are you'

### Character Casing

In [17]:
def character_casing(text):
    lower_text = text.lower()
    return lower_text

In [18]:
character_casing("STARBUCKS Starbucks")

'starbucks starbucks'

### Apply Preprocessing Functions

In [19]:
text_hotel[:5]

,Review
0,fantastic service large hotel caters business ...
1,"great hotel modern hotel good location, locate..."
2,3 star plus glasgowjust got 30th november 4 da...
3,nice stayed hotel nov 19-23. great little bout...
4,great place wonderful hotel ideally located me...


In [20]:
text_hotel["Review"] = text_hotel["Review"].apply(sp_chara_cleaning)
text_hotel["Review"] = text_hotel["Review"].apply(character_casing)

In [21]:
# show text after preprocessing
text_hotel[:5]

,Review
0,fantastic service large hotel caters business ...
1,great hotel modern hotel good location located...
2,3 star plus glasgowjust got 30th november 4 da...
3,nice stayed hotel nov 1923 great little boutiq...
4,great place wonderful hotel ideally located me...


In [22]:
text_scifi[:20]

' A chat with the edi'

In [23]:
text_scifi = sp_chara_cleaning(text_scifi)
text_scifi = character_casing(text_scifi)
# show text after preprocessing
text_scifi[:20]

' a chat with the edi'

### Create Dataset For Training

In [24]:
def create_triagrams(i_text, context_size=2):
    i_context_tensor = []
    i_target_tensor = []
    for i in range(context_size, len(i_text) - context_size):
        i_contexts = [i_text[j] for j in range(i - context_size, i + context_size + 1) if i != j]
        i_target = i_text[i]
        i_context_tensor.append(i_contexts)
        i_target_tensor.append(i_target)
    i_target_tensor = torch.LongTensor(i_target_tensor)
    i_context_tensor = torch.LongTensor(i_context_tensor)
    return i_context_tensor, i_target_tensor

#### SCIFI DATASET

In [25]:
words = text_scifi.split()
vocab_scifi = set(words)
word_to_ix = {word: i for i, word in enumerate(vocab_scifi)}
i_text = [word_to_ix[w] for w in words]
i_context_scifi, i_target_scifi = create_triagrams(i_text, 2)



In [26]:
len(words), len(vocab_scifi), i_target_scifi.shape, i_context_scifi.shape

(7612053, 132702, torch.Size([7612049]), torch.Size([7612049, 4]))

In [ ]:

# def create_triagrams(i_text, context_size, i_data):

#     for i in range(context_size, len(i_text) - context_size):
#         idxs = [i_text[j] for j in range(i - context_size, i + context_size + 1)]
#         idxs.pop(len(idxs) // 2)
#         i_target = i_text[i]
#         i_data.append((torch.tensor(idxs, dtype=torch.long), torch.tensor(i_target, dtype=torch.long)))

#     return i_data

#### HOTEL DATASET

In [27]:
words = ' '.join(text_hotel['Review']).split()
vocab_hotel = set(words)
word_to_ix = {word: i for i, word in enumerate(vocab_hotel)}
i_text = [word_to_ix[w] for w in words]

In [28]:
i_context_hotel_2, i_target_hotel_2 = create_triagrams(i_text, 2)
i_context_hotel_5, i_target_hotel_5 = create_triagrams(i_text, 5)

In [29]:
len(words), len(vocab_hotel)

(1051086, 51839)

In [30]:
i_context_hotel_2.shape, i_target_hotel_2.shape, i_context_hotel_5.shape, i_target_hotel_5.shape

(torch.Size([1051082, 4]),
 torch.Size([1051082]),
 torch.Size([1051076, 10]),
 torch.Size([1051076]))

## Train Model

In [31]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [32]:
torch.cuda.is_available()

True

In [35]:
import time

### CBOW 2 with Hotel Dataset

In [ ]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 50
losses = []
loss_function = nn.NLLLoss()
CBOW2_hotel_model = CBOW(len(vocab_hotel), EMBEDDING_DIM, CONTEXT_SIZE).to(device)
optimizer = optim.SGD(CBOW2_hotel_model.parameters(), lr=0.001)

for epoch in range(12):
    total_loss = 0
    start_time = time.time()
    for i in range(len(i_target_hotel_2)):
        i_context = i_context_hotel_2[i].to(device)
        i_target = i_target_hotel_2[i].to(device)


        # torch accumulates gradients. Before passing in a
        # new instance, zero out the gradients from the old
        # instance
        CBOW2_hotel_model.zero_grad()

        # Run the forward pass, getting log probabilities over next
        # words
        log_probs = CBOW2_hotel_model(i_context)

        # n probs, 1 target idx
        # Compute loss function.
        loss = loss_function(log_probs, i_target.unsqueeze(dim=0))

        # Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    print(f"Epoch: {epoch}, AVG loss: {total_loss/len(i_target_hotel_2):.4f}, \
Time: {time.time() - start_time:.2f}s")
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

Epoch: 0, AVG loss: 7.8995, Time: 1571.76s


In [ ]:
torch.save(CBOW2_hotel_model.state_dict(), 'CBOW2_hotel_model.pth')

### CBOW 5 with Hotel Dataset

In [ ]:
CONTEXT_SIZE = 5
EMBEDDING_DIM = 50
losses = []
loss_function = nn.NLLLoss()
CBOW5_hotel_model = CBOW(len(vocab_hotel), EMBEDDING_DIM, CONTEXT_SIZE).to(device)
optimizer = optim.SGD(CBOW5_hotel_model.parameters(), lr=0.001)

for epoch in range(12):
    total_loss = 0
    start_time = time.time()
    for i in range(len(i_target_hotel_5)):
        i_context = i_context_hotel_5[i].to(device)
        i_target = i_target_hotel_5[i].to(device)


        # torch accumulates gradients. Before passing in a
        # new instance, zero out the gradients from the old
        # instance
        CBOW5_hotel_model.zero_grad()

        # Run the forward pass, getting log probabilities over next
        # words
        log_probs = CBOW5_hotel_model(i_context)

        # n probs, 1 target idx
        # Compute loss function.
        loss = loss_function(log_probs, i_target.unsqueeze(dim=0))

        # Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    print(f"Epoch: {epoch}, AVG loss: {total_loss/len(i_target_hotel_5):.4f}, \
Time: {time.time() - start_time:.2f}s")
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

In [ ]:
torch.save(CBOW5_hotel_model.state_dict(), 'CBOW5_hotel_model.pth')

### CBOW 2 with Scifi Dataset

In [34]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 50
losses = []
loss_function = nn.NLLLoss()
CBOW2_scifi_model = CBOW(len(vocab_scifi), EMBEDDING_DIM, CONTEXT_SIZE).to(device)
optimizer = optim.SGD(CBOW2_scifi_model.parameters(), lr=0.001)

for epoch in range(3):
    total_loss = 0
    for i in range(len(i_target_scifi)):
        i_context = i_context_scifi[i].to(device)
        i_target = i_target_scifi[i].to(device)


        # torch accumulates gradients. Before passing in a
        # new instance, zero out the gradients from the old
        # instance
        CBOW2_scifi_model.zero_grad()

        # Run the forward pass, getting log probabilities over next
        # words
        log_probs = CBOW2_scifi_model(i_context)

        # n probs, 1 target idx
        # Compute loss function.
        loss = loss_function(log_probs, i_target.unsqueeze(dim=0))

        # Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    print(f"Epoch: {epoch}, AVG loss: {total_loss/len(i_target_scifi):.4f}, \
Time: {time.time() - start_time:.2f}s")
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

Epoch: 0, AVG loss: 0.0112
Epoch: 1, AVG loss: 0.0111
Epoch: 2, AVG loss: 0.0109
[11767.49444103241, 11629.643478393555, 11484.67548084259]


In [ ]:
torch.save(CBOW2_scifi_model.state_dict(), 'CBOW2_scifi_model.pth')